In [ ]:
# Load model with pretrained weights
from super_gradients.training import models
from super_gradients.common.object_names import Models

# model = models.get(Models.YOLO_NAS_M, pretrained_weights="coco")

In [ ]:
from super_gradients import training

In [ ]:
# PP_YOLOE_S
# model = models.get(Models.PP_YOLOE_S, pretrained_weights="coco", num_classes=80)

# Get pretrained model from super-gradients repository.

model = models.get(Models.PP_YOLOE_S, pretrained_weights="coco")
# models.convert_to_onnx(model=model, input_shape=(3,640,640), out_path="/workspace/pp-yoloe-s.onnx")


# model = models.get(Models.PP_LITE_B_SEG50, pretrained_weights="cityscapes")

In [ ]:
training.models.model_factory.get_architecture(Models.PP_YOLOE_S, model.get_)

In [ ]:
# models.convert_to_onnx(model, './pp_yoloe_s.onnx', (640, 640, 3))

In [ ]:
image_path = 'https://cdn.shopify.com/s/files/1/0084/8242/2848/files/why_us_img.jpg?v=1633587264'
model.predict(image_path).show()

In [ ]:
# model = model.to('cpu')
# model.predict_webcam('/dev/video/0').show()

In [ ]:
from torchinfo import summary

summary(model=model,
        input_size=[5, 3, 640, 640],
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width=20,
        row_settings=['var_names']
        )

In [ ]:
import requests
from PIL import Image
from io import BytesIO

# Image can be both uploaded to colab server or by a direct URL
image_path = "https://www.gulfplaceon30a.com/wp-content/uploads/2020/01/yolo1.jpg"

# Load image from url
response = requests.get(image_path)

# Get PIL image
image = Image.open(BytesIO(response.content))

In [ ]:
import torchvision.transforms as transforms
import torch

# Prepare preprcoess transformations
# We resize to [640, 640] by COCO's dataset default, which the model was pretrained on.
preprocess = transforms.Compose([
    transforms.Resize([640, 640]),
    transforms.PILToTensor()
])

# Run preprocess on image. unsqueeze for [Batch x Channels x Width x Height] format
transformed_image = preprocess(image).float().unsqueeze(0)
# print(transformed_image)

In [ ]:
model.eval()

# Predict using SG model
with torch.no_grad():
  raw_predictions = model(transformed_image)

In [ ]:
print(raw_predictions[0][0].shape)
print(raw_predictions[0][1].shape)

# print(raw_predictions[1][0].shape)
# print(raw_predictions[1][1].shape)

In [ ]:
from super_gradients.training.models.detection_models.yolo_base import YoloPostPredictionCallback
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

# Callback uses NMS, confidence threshold
# predictions = YoloPostPredictionCallback(conf=0.1, iou=0.4)(raw_predictions)[0].numpy()

# nx6 (x1, y1, x2, y2, confidence, class) => [N, 6]
predictions = PPYoloEPostPredictionCallback(0.1, 0.4, 5, 1000)(raw_predictions, 'cpu')

In [ ]:
predictions[0].shape

In [ ]:
import matplotlib.pyplot as plt

# Visualize results
boxes = predictions[:, 0:4]
plt.figure(figsize=(10, 10))
plt.plot(boxes[:, [0, 2, 2, 0, 0]].T, boxes[:, [1, 1, 3, 3, 1]].T, '.-')
plt.imshow(image.resize([640, 640]))
plt.show()